In [3]:
import pandas as pd
from scipy.stats import poisson
from sqlalchemy import and_

from models import Player, TradPlayerStats, AdvPlayerStats, Game
from data_manager import DataManager
dm = DataManager()
session = dm.get_session()


In [69]:
def estimate_probability_poisson(data, stat, n):
    mean = data[stat].mean()
    probability = 1 - poisson.cdf(n, mean)
    return probability

def american_to_decimal(american_odds):
    """Convert American odds to decimal odds."""
    if american_odds > 0:
        return 1 + (american_odds / 100)
    else:
        return 1 + (100 / abs(american_odds))

def calculate_parlay_odds(american_odds_list):
    """Calculate the combined decimal odds for a parlay given a list of American odds."""
    decimal_odds = [american_to_decimal(odds) for odds in american_odds_list]
    
    # Calculate combined odds for the parlay
    combined_odds = 1
    for odds in decimal_odds:
        combined_odds *= odds
        
    return combined_odds

# Query to join four tables with correct join conditions and multiple filters
def get_and_save_player_data(player_id, player_name):
    data = session.query(
        Player,
        TradPlayerStats,
        AdvPlayerStats,
        Game
    ).join(Game, TradPlayerStats.game_id == Game.id)\
    .join(Player, TradPlayerStats.player_id == Player.id)\
    .join(AdvPlayerStats, and_(TradPlayerStats.game_id == AdvPlayerStats.game_id, 
                                TradPlayerStats.player_id == AdvPlayerStats.player_id))\
    .filter(
        (TradPlayerStats.player_id == player_id)# &
        #  (Game.season_type == "Playoffs")
    ).all()

    # Convert the query result to a DataFrame
    data_list = []
    for player, trad_stats, adv_stats, game in data:
        row = {
            'player_name': player.name,
            'player_position': player.position,
            'minutes': trad_stats.minutes,
            'points': trad_stats.pts,
            'rebounds': trad_stats.reb,
            'assists': trad_stats.ast,
            'efg': adv_stats.efg_pct,
            'fg3a': trad_stats.fg3a,
            'fg3m': trad_stats.fg3m,
            'fg3_pct': trad_stats.fg3_pct,
            'fga': trad_stats.fga,
            'fgm': trad_stats.fgm,
            'fta': trad_stats.fta,
            'ft_pct': trad_stats.ft_pct, 
            'steals': trad_stats.stl,
            'blocks': trad_stats.blk,
            'date': game.date,

        }
        data_list.append(row)

    data_df = pd.DataFrame(data_list)

    data_df.to_csv(f"data_pile/{player_name}.csv")
    return data_df


def get_player_id(player_name):
    player = session.query(Player).filter(Player.name==player_name).all()[0]
    player_id = player.id
    return player_id


def calculate_ev(probability, decimal_odds, bet_amount):
    """
    Calculate the expected value (EV) of a bet.

    Args:
        probability (float): The probability of the event occurring (between 0 and 1).
        decimal_odds (float): The decimal odds for the bet.
        bet_amount (float): The amount wagered.

    Returns:
        float: The expected value (EV) of the bet.
    """
    payout = decimal_odds * bet_amount
    ev = (probability * payout) - bet_amount
    return ev


def implied_probability(decimal_odds):
    """Convert decimal odds to implied probability."""
    return 1 / decimal_odds


def estimate_probability_poisson_under(data, stat, n):
    mean = data[stat].mean()
    probability = poisson.cdf(n, mean)  # Calculate P(X <= n)
    return probability


def get_prop_probability(player_name, stat, prop_threshold, last_n_games=25, bet_type="over"):
    player_id = get_player_id(player_name)
    data = get_and_save_player_data(player_id, player_name).sort_values(by='date', ascending=False).head(last_n_games).copy()
    print(data.head())
    if bet_type == "over":
        return estimate_probability_poisson(data, stat, prop_threshold)
    elif bet_type == "under":
        return estimate_probability_poisson_under(data, stat, prop_threshold)
    else:
        raise ValueError("Invalid bet type. Use 'over' or 'under'.")
    

def analyze_bet(player_name, stat, threshold, odds, last_n_games, bet_type):
    probability = get_prop_probability(player_name, stat, threshold, last_n_games=last_n_games, bet_type=bet_type)
    odds = american_to_decimal(odds)
    house_probability = implied_probability(odds)
    ev = calculate_ev(probability, odds, 5)
    print(f"house_probability: {house_probability}")
    print(f"our probability: {probability}")
    print(f"odds: {odds}")
    print(f"ev: {ev}")
    return probability, odds


def calculate_combined_probability(probabilities):
    """Calculate the combined probability for a parlay."""
    combined_probability = 1
    for prob in probabilities:
        combined_probability *= prob
    return combined_probability


def calculate_combined_odds(decimal_odds_list):
    """Calculate the combined decimal odds for a parlay."""
    combined_odds = 1
    for odds in decimal_odds_list:
        combined_odds *= odds
    return combined_odds


def analyze_parlay(probabilities, odds_list, bet_amount):
    """
    Analyze a parlay bet and print the combined probability, combined odds, and expected value.

    Args:
        probabilities (list of float): The probabilities of individual bets.
        american_odds_list (list of int): The American odds for individual bets.
        bet_amount (float): The amount wagered.
    """
    combined_probability = calculate_combined_probability(probabilities)
    combined_odds = calculate_combined_odds(odds_list)
    ev = calculate_ev(combined_probability, combined_odds, bet_amount)
    
    print(f"Combined Probability: {combined_probability:.4f}")
    print(f"Combined Odds: {combined_odds:.2f}")
    print(f"Expected Value (EV): ${ev:.2f}")

    return ev

class Prop:
    def __init__(self, name, stat, threshold, probability, odds, bet_type):
        self.name = name
        self.stat = stat
        self.n = threshold
        self.probability = probability
        self.odds = odds
        self.bet_type = bet_type
        

def analyze_parlay_list(parlay_list, bet_amount):
    probabilities = [prop.probability for prop in parlay_list]
    odds = [prop.odds for prop in parlay_list]
    return analyze_parlay(probabilities, odds, bet_amount)
    

In [ ]:
for prop in parlay:
    analyze_bet()

In [124]:
player_name = "Nikola Jokic"
stat = "points"
threshold = 29.5
odds = -120
# bet_type = "over"
bet_type = "under"

probability, odds = analyze_bet(player_name, stat, threshold, odds, last_n_games = 25, bet_type = bet_type)



     player_name player_position  minutes  points  rebounds  assists    efg  \
88  Nikola Jokic               C     41.0      40         7       13  0.727   
1   Nikola Jokic               C     39.0      35         7        7  0.596   
0   Nikola Jokic               C     38.0      24        14        9  0.583   
5   Nikola Jokic               C     38.0      16        16        8  0.385   
3   Nikola Jokic               C     40.0      32         8        9  0.480   

    fg3a  fg3m  fg3_pct  fga  fgm  fta  ft_pct  steals  blocks        date  
88     3     2    0.667   22   15    9   0.889       2       1  2024-05-14  
1      5     1    0.200   26   15    6   0.667       3       1  2024-05-12  
0      3     1    0.333   18   10    4   0.750       3       3  2024-05-10  
5      1     0    0.000   13    5    6   1.000       0       1  2024-05-06  
3      9     2    0.222   25   11    8   1.000       3       0  2024-05-04  
house_probability: 0.5454545454545454
our probability: 0.587522

In [110]:
parlay = []

In [82]:
[print(prop.name, prop.stat, prop.n, prop.bet_type) for prop in parlay]

Luguentz Dort blocks 0.5 under


[None]

In [89]:
parlay.pop()
[print(prop.name, prop.stat, prop.n, prop.bet_type) for prop in parlay]

Luguentz Dort blocks 0.5 under
Derrick Jones Jr. fg3m 0.5 under


[None, None]

In [122]:
prop_to_add = Prop(player_name, stat, threshold, probability, odds, bet_type)
parlay.append(prop_to_add)
[print(prop.name, prop.stat, prop.n, prop.bet_type) for prop in parlay]



Kyrie Irving points 21.5 over
Shai Gilgeous-Alexander points 32.5 under
Luguentz Dort steals 1.5 under


[None, None, None]

In [123]:
[print(prop.name, prop.stat, prop.n, prop.bet_type) for prop in parlay]
print(analyze_parlay_list(parlay, 5))

Kyrie Irving points 21.5 over
Shai Gilgeous-Alexander points 32.5 under
Luguentz Dort steals 1.5 under
Combined Probability: 0.4782
Combined Odds: 4.98
Expected Value (EV): $6.90
6.904746624032631
